"What makes unlearning hard and what to do about it" (Zhao et al., NeurIPS 2024) 논문의 Table 5a와 유사한 결과를 재현하는 코드임

이 코드는 논문에 기술된 실험 절차, 특히 ES(Entanglement Score)에 따른 데이터 분할, 각종 언러닝(Unlearning) 방법 적용, 그리고 ToW 및 예측 차이율 계산 과정을 모두 포함하고 있음

L1-sparse, SalUn 등 일부 복잡한 언러닝 알고리즘은 논문의 핵심 아이디어를 바탕으로 단순화하여 구현함

In [4]:
!nvidia-smi

Sun Jul 20 15:55:37 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:04:00.0 Off |                    0 |
| N/A   40C    P0              34W / 250W |  12422MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
import torch

# Set device (GPU if available)
DEVICE_NUM = 6
ADDITIONAL_GPU = 1

if torch.cuda.is_available():
    torch.cuda.set_device(DEVICE_NUM)
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}:{DEVICE_NUM}")

INFO: Using device - cuda:6


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Subset, Dataset
import numpy as np
import pandas as pd
from scipy import stats
import time
import copy
import itertools
import os
# from torch_influence import WoodFisherInfluenceComputer

# ===================================================================
# 1. 실험 환경 설정
# ===================================================================
CONFIG = {
    "run_training": True,  # True: 훈련 및 저장 실행, False: 저장된 파일 로드하여 평가만 실행
    "model_save_dir": "../saved_models", 
    "num_runs": 3,
    "epochs": 30,
    "unlearn_epochs": 10,
    "batch_size": 256,
    "lr": 0.1,
    "unlearn_lr": 0.01,
    "unlearn_lr_neggrad": 1e-4, # NegGrad는 작은 LR 사용
    "momentum": 0.9,
    "weight_decay": 5e-4,
    "forget_set_size": 3000,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "l1_lambda": 1e-5,
    "neggrad_plus_alpha": 0.2,
    "salun_sparsity": 0.5,
}

print(f"Using device: {CONFIG['device']}")

# ===================================================================
# 2. 모델 및 데이터 관련 헬퍼 함수
# ===================================================================
def get_model():
    """ResNet-18 모델 생성"""
    model = models.resnet18(weights=None, num_classes=10)
    return model.to(CONFIG['device'])

def train_model(model, train_loader, epochs, lr, is_unlearning=False):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=CONFIG['momentum'], weight_decay=CONFIG['weight_decay'])
    if not is_unlearning:
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    model.train()
    for epoch in range(epochs):
        epoch_start_time = time.time() # 에포크 시작 시간 기록
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(CONFIG['device']), targets.to(CONFIG['device'])
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        if not is_unlearning:
            scheduler.step()
        
        # 에포크별 진행 상황 출력
        epoch_end_time = time.time()
        print(f"    Epoch {epoch+1}/{epochs} completed in {epoch_end_time - epoch_start_time:.2f}s")

def evaluate_model(model, loader):
    """모델 정확도 평가"""
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(CONFIG['device']), targets.to(CONFIG['device'])
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    return 100 * correct / total

# ===================================================================
# 3. ES(Entanglement Score) 기반 데이터 분할 
# ===================================================================
def create_es_partitions(original_model, train_dataset):
    """
    논문 A.3 절차에 따라 ES 파티션을 생성
    1. 원본 모델의 임베딩 추출
    2. 전체 임베딩의 중심(centroid) 계산
    3. 각 데이터와 중심 간의 거리 계산 후 정렬
    4. 거리가 가장 먼 데이터부터 low, medium, high ES 그룹 생성
    """
    print("Creating ES partitions...")
    start_time = time.time() 
    # 1. 임베딩 추출
    embedding_extractor = nn.Sequential(*list(original_model.children())[:-1])
    embedding_extractor.eval()
    
    all_embeddings = []
    loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=False)
    # 임베딩 추출 진행 상황 출력
    print("  Extracting embeddings from original model...")
    for i, (inputs, _) in enumerate(loader):
        if (i + 1) % 40 == 0:
            print(f"    Batch {i+1}/{len(loader)}")
        with torch.no_grad():
            inputs = inputs.to(CONFIG['device'])
            embeddings = embedding_extractor(inputs).squeeze()
            all_embeddings.append(embeddings.cpu())
    all_embeddings = torch.cat(all_embeddings, dim=0)

    centroid = all_embeddings.mean(dim=0)
    distances = torch.sum((all_embeddings - centroid) ** 2, dim=1)
    sorted_indices = torch.argsort(distances, descending=True).numpy()

    fs_size = CONFIG['forget_set_size']
    partitions = {
        "Low ES": sorted_indices[:fs_size],
        "Medium ES": sorted_indices[fs_size : 2 * fs_size],
        "High ES": sorted_indices[2 * fs_size : 3 * fs_size],
    }
    end_time = time.time() 
    print(f"ES partitions created in {end_time - start_time:.2f}s.")
    return partitions

# ===================================================================
# 4. 언러닝(Unlearning) 알고리즘 구현
# ===================================================================
class RelabelDataset(Dataset):
    """Random-label과 SalUn에서 레이블을 변경하기 위한 커스텀 데이터셋"""
    def __init__(self, original_dataset, num_classes=10):
        self.original_dataset = original_dataset
        self.num_classes = num_classes
        # 원본 레이블과 다른 랜덤 레이블 생성
        self.new_labels = [torch.randint(0, num_classes, (1,)).item() for _ in range(len(original_dataset))]
        
    def __len__(self):
        return len(self.original_dataset)

    def __getitem__(self, idx):
        image, original_label = self.original_dataset[idx]
        new_label = self.new_labels[idx]
        # 원본 레이블과 같지 않도록 보장
        while new_label == original_label:
            new_label = torch.randint(0, self.num_classes, (1,)).item()
        return image, new_label

def unlearn_finetune(original_model, retain_loader, config):
    """Fine-tune: 원본 모델을 retain set으로 추가 학습"""
    unlearned_model = copy.deepcopy(original_model)
    train_model(unlearned_model, retain_loader, config['unlearn_epochs'], config['unlearn_lr'], is_unlearning=True)
    return unlearned_model

def unlearn_neggrad(original_model, forget_loader, config):
    """NegGrad: Forget set에 대해 loss를 최대화 (gradient ascent)"""
    unlearned_model = copy.deepcopy(original_model)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(unlearned_model.parameters(), lr=config['unlearn_lr_neggrad'])
    
    unlearned_model.train()
    for _ in range(config['unlearn_epochs']):
        for inputs, targets in forget_loader:
            inputs, targets = inputs.to(config['device']), targets.to(config['device'])
            optimizer.zero_grad()
            outputs = unlearned_model(inputs)
            loss = -criterion(outputs, targets) # Loss를 최대화하기 위해 음수 사용
            loss.backward()
            optimizer.step()
    return unlearned_model

def unlearn_l1_sparse(original_model, retain_loader, config):
    """L1-sparse: 리테인셋에 L1 패널티를 주며 파인튜닝"""
    unlearned_model = copy.deepcopy(original_model)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(unlearned_model.parameters(), lr=config['unlearn_lr'], momentum=config['momentum'])
    
    unlearned_model.train()
    for _ in range(config['unlearn_epochs']):
        for inputs, targets in retain_loader:
            inputs, targets = inputs.to(config['device']), targets.to(config['device'])
            optimizer.zero_grad()
            outputs = unlearned_model(inputs)
            
            # L1 패널티 계산
            l1_penalty = 0.
            for param in unlearned_model.parameters():
                l1_penalty += torch.abs(param).sum()
            
            loss = criterion(outputs, targets) + config['l1_lambda'] * l1_penalty
            loss.backward()
            optimizer.step()
    return unlearned_model

def unlearn_neggrad_plus(original_model, retain_loader, forget_loader, config):
    """NegGrad+: Retain loss 최소화와 Forget loss 최대화를 동시에 수행"""
    unlearned_model = copy.deepcopy(original_model)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(unlearned_model.parameters(), lr=config['unlearn_lr'])

    unlearned_model.train()
    for _ in range(config['unlearn_epochs']):
        # 데이터로더 길이가 다를 수 있으므로 itertools.cycle 사용
        retain_iter = iter(itertools.cycle(retain_loader))
        for forget_inputs, forget_targets in forget_loader:
            retain_inputs, retain_targets = next(retain_iter)
            
            retain_inputs, retain_targets = retain_inputs.to(config['device']), retain_targets.to(config['device'])
            forget_inputs, forget_targets = forget_inputs.to(config['device']), forget_targets.to(config['device'])
            
            optimizer.zero_grad()
            
            # Retain Loss (최소화)
            retain_outputs = unlearned_model(retain_inputs)
            loss_retain = criterion(retain_outputs, retain_targets)
            
            # Forget Loss (최대화)
            forget_outputs = unlearned_model(forget_inputs)
            loss_forget = -criterion(forget_outputs, forget_targets)
            
            # 두 Loss를 결합
            loss = loss_retain + config['neggrad_plus_alpha'] * loss_forget
            loss.backward()
            optimizer.step()
    return unlearned_model

def unlearn_random_label(original_model, forget_set, config):
    """Random-label: Forget set의 레이블을 랜덤으로 바꾸어 학습"""
    unlearned_model = copy.deepcopy(original_model)
    relabel_dataset = RelabelDataset(forget_set)
    relabel_loader = DataLoader(relabel_dataset, batch_size=config['batch_size'], shuffle=True)
    
    # 일반적인 파인튜닝과 동일하게 학습
    train_model(unlearned_model, relabel_loader, config['unlearn_epochs'], config['unlearn_lr'], is_unlearning=True)
    return unlearned_model

def unlearn_salun(original_model, forget_set, config):
    """SalUn: Forget set에 대해 중요한(salient) 파라미터만 랜덤 레이블로 학습"""
    unlearned_model = copy.deepcopy(original_model)
    
    # 1. Saliency 계산 (Gradient 기반)
    saliency = [torch.zeros_like(p) for p in unlearned_model.parameters()]
    criterion = nn.CrossEntropyLoss()
    forget_loader = DataLoader(forget_set, batch_size=config['batch_size'])

    for inputs, targets in forget_loader:
        inputs, targets = inputs.to(config['device']), targets.to(config['device'])
        unlearned_model.zero_grad()
        outputs = unlearned_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        for i, param in enumerate(unlearned_model.parameters()):
            if param.grad is not None:
                saliency[i] += param.grad.abs()

    # 2. Saliency가 높은 파라미터를 찾기 위한 마스크 생성
    flat_saliency = torch.cat([s.flatten() for s in saliency])
    k = int(len(flat_saliency) * config['salun_sparsity'])
    threshold, _ = torch.kthvalue(flat_saliency, k)
    masks = [(s > threshold).float() for s in saliency]

    # 3. 랜덤 레이블 데이터셋으로 Salient 파라미터만 학습
    relabel_dataset = RelabelDataset(forget_set)
    relabel_loader = DataLoader(relabel_dataset, batch_size=config['batch_size'], shuffle=True)
    optimizer = optim.SGD(unlearned_model.parameters(), lr=config['unlearn_lr'], momentum=config['momentum'])
    
    unlearned_model.train()
    for _ in range(config['unlearn_epochs']):
        for inputs, targets in relabel_loader:
            inputs, targets = inputs.to(config['device']), targets.to(config['device'])
            optimizer.zero_grad()
            outputs = unlearned_model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            
            # 마스크를 적용하여 salient 파라미터의 gradient만 유지
            for i, param in enumerate(unlearned_model.parameters()):
                if param.grad is not None:
                    param.grad *= masks[i]
            
            optimizer.step()
            
    return unlearned_model

def unlearn_influence(original_model, forget_indices, train_dataset, config):
    """
    Influence Unlearning: influence 함수를 사용해 forget set의 영향을 추정하고 제거함
    이 방법은 재훈련 없이 파라미터의 변화를 근사적으로 계산함
    """
    unlearned_model = copy.deepcopy(original_model)
    criterion = nn.CrossEntropyLoss()
    
    # 1. InfluenceComputer 초기화
    # 이 과정에서 내부적으로 Hessian-vector product 계산 준비
    # 이는 계산 비용이 매우 높을 수 있음
    computer = WoodFisherInfluenceComputer(
        model=unlearned_model,
        criterion=criterion,
        train_dataset=train_dataset,
        device=config['device'],
        damp=1e-6, # 계산 안정성을 위한 하이퍼파라미터
        gnh=True   # Generalized Gauss-Newton 행렬 사용
    )

    # 2. Forget set의 영향 계산
    # forget_indices에 해당하는 데이터가 모델 파라미터에 미친 영향을 추정
    param_changes = computer.compute_influence_on_parameters(
        train_indices=forget_indices
    )
    
    # 3. 모델 파라미터 업데이트
    # 원본 모델의 파라미터에서 계산된 영향을 빼서 업데이트
    with torch.no_grad():
        for i, param in enumerate(unlearned_model.parameters()):
            param.sub_(param_changes[i] * config['influence_alpha']) # alpha는 스케일링 팩터

    return unlearned_model

# ===================================================================
# 5. 결과 지표 계산 함수
# ===================================================================
def calculate_tow(unlearned_accs, retrained_accs):
    """ToW (Tug-of-War) 점수를 계산"""
    la = lambda acc1, acc2: abs(acc1 - acc2) / 100.0
    la_s = la(unlearned_accs['forget'], retrained_accs['forget'])
    la_r = la(unlearned_accs['retain'], retrained_accs['retain'])
    la_t = la(unlearned_accs['test'], retrained_accs['test'])
    return (1 - la_s) * (1 - la_r) * (1 - la_t)

def calculate_prediction_diff(unlearned_model, retrained_model, full_dataset_loader):
    """두 모델의 예측이 다른 샘플의 비율을 계산"""
    unlearned_model.eval()
    retrained_model.eval()
    diff_count = 0
    total_count = 0
    with torch.no_grad():
        for inputs, _ in full_dataset_loader:
            inputs = inputs.to(CONFIG['device'])
            unlearned_preds = torch.argmax(unlearned_model(inputs), 1)
            retrained_preds = torch.argmax(retrained_model(inputs), 1)
            diff_count += (unlearned_preds != retrained_preds).sum().item()
            total_count += inputs.size(0)
    return (diff_count / total_count) * 100

# ===================================================================
# 6. 메인 실험 루프 
# ===================================================================
def main():
    # 모델 저장을 위한 디렉토리 생성
    save_dir = CONFIG["model_save_dir"]
    if CONFIG["run_training"]:
        os.makedirs(save_dir, exist_ok=True)

    # 데이터셋 로드
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False)
    
    full_train_eval_loader = DataLoader(
        datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_test),
        batch_size=CONFIG['batch_size'], shuffle=False
    )
    
    all_methods = ["Original", "Fine-tune", "L1-sparse", "NegGrad", "NegGrad+", "SalUn", "Random-label"]
    results = {
        "ToW": {method: {es: [] for es in ["Low ES", "Medium ES", "High ES"]} for method in all_methods},
        "Pred_Diff": {method: {es: [] for es in ["Low ES", "Medium ES", "High ES"]} for method in all_methods}
    }

    for run in range(CONFIG['num_runs']):
        print(f"\n{'='*20} Starting Run {run+1}/{CONFIG['num_runs']} {'='*20}")

        # 훈련 모드와 평가 모드 분리
        original_model = get_model()
        original_model_path = os.path.join(save_dir, f"run_{run}_original_model.pth")
        partitions_path = os.path.join(save_dir, f"run_{run}_es_partitions.pth")

        if CONFIG["run_training"]:
            print("\n[TRAINING MODE] Step 1/5: Training original model...")
            run_start_time = time.time()
            train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True)
            train_model(original_model, train_loader, CONFIG['epochs'], CONFIG['lr'])
            torch.save(original_model.state_dict(), original_model_path)
            print(f"Original model trained and saved in {time.time() - run_start_time:.2f}s")
            
            es_partitions = create_es_partitions(original_model, train_dataset)
            torch.save(es_partitions, partitions_path)
        else:
            print("\n[EVALUATION MODE] Loading pre-trained original model and ES partitions...")
            original_model.load_state_dict(torch.load(original_model_path, map_location=CONFIG['device']))
            es_partitions = torch.load(partitions_path)

        for es_level, forget_indices in es_partitions.items():
            print(f"\n--- Processing ES Level: {es_level} ---")
            
            all_indices = np.arange(len(train_dataset))
            retain_indices = np.setdiff1d(all_indices, forget_indices, assume_unique=True)
            
            retain_set = Subset(train_dataset, retain_indices)
            forget_set = Subset(train_dataset, forget_indices)
            
            retain_loader = DataLoader(retain_set, batch_size=CONFIG['batch_size'], shuffle=True)
            retain_eval_loader = DataLoader(retain_set, batch_size=CONFIG['batch_size'], shuffle=False)
            forget_loader = DataLoader(forget_set, batch_size=CONFIG['batch_size'], shuffle=False)

            retrained_model = get_model()
            retrained_model_path = os.path.join(save_dir, f"run_{run}_{es_level.replace(' ', '')}_retrained.pth")

            if CONFIG["run_training"]:
                print(f"\n[TRAINING MODE] Step 2/5: Training retrained model for {es_level}...")
                retrain_start_time = time.time()
                train_model(retrained_model, retain_loader, CONFIG['epochs'], CONFIG['lr'])
                torch.save(retrained_model.state_dict(), retrained_model_path)
                print(f"Retrained model trained and saved in {time.time() - retrain_start_time:.2f}s")
            else:
                print(f"\n[EVALUATION MODE] Loading retrained model for {es_level}...")
                retrained_model.load_state_dict(torch.load(retrained_model_path, map_location=CONFIG['device']))
            
            print("\n[Step 3/5] Evaluating retrained model...")
            retrained_accs = {
                'retain': evaluate_model(retrained_model, retain_eval_loader),
                'forget': evaluate_model(retrained_model, forget_loader),
                'test': evaluate_model(retrained_model, test_loader)
            }
            print(f"  Retrained Accs -> Retain: {retrained_accs['retain']:.2f}%, Forget: {retrained_accs['forget']:.2f}%, Test: {retrained_accs['test']:.2f}%")
            
            unlearning_methods = {
                "Original": lambda: copy.deepcopy(original_model),
                "Fine-tune": lambda: unlearn_finetune(original_model, retain_loader, CONFIG),
                "L1-sparse": lambda: unlearn_l1_sparse(original_model, retain_loader, CONFIG),
                "NegGrad": lambda: unlearn_neggrad(original_model, forget_loader, CONFIG),
                "NegGrad+": lambda: unlearn_neggrad_plus(original_model, retain_loader, forget_loader, CONFIG),
                "SalUn": lambda: unlearn_salun(original_model, forget_set, CONFIG),
                "Random-label": lambda: unlearn_random_label(original_model, forget_set, CONFIG),
            }
            
            print("\n[Step 4/5] Applying and evaluating unlearning methods...")
            for method_name, method_fn in unlearning_methods.items():
                unlearned_model_path = os.path.join(save_dir, f"run_{run}_{es_level.replace(' ', '')}_{method_name}_unlearned.pth")
                
                if CONFIG["run_training"]:
                    print(f"  > [TRAINING MODE] Applying method: {method_name}...")
                    method_start_time = time.time()
                    # Original은 훈련이 없으므로 바로 저장
                    unlearned_model = method_fn()
                    torch.save(unlearned_model.state_dict(), unlearned_model_path)
                    print(f"    Method {method_name} applied and saved in {time.time() - method_start_time:.2f}s")
                else:
                    print(f"  > [EVALUATION MODE] Loading unlearned model for method: {method_name}...")
                    unlearned_model = get_model() 
                    unlearned_model.load_state_dict(torch.load(unlearned_model_path, map_location=CONFIG['device']))

                unlearned_accs = {
                    'retain': evaluate_model(unlearned_model, retain_eval_loader),
                    'forget': evaluate_model(unlearned_model, forget_loader),
                    'test': evaluate_model(unlearned_model, test_loader)
                }
                
                tow_score = calculate_tow(unlearned_accs, retrained_accs)
                pred_diff = calculate_prediction_diff(unlearned_model, retrained_model, full_train_eval_loader)

                results["ToW"][method_name][es_level].append(tow_score)
                results["Pred_Diff"][method_name][es_level].append(pred_diff)
                
                print(f"    - Unlearned Accs -> Retain: {unlearned_accs['retain']:.2f}%, Forget: {unlearned_accs['forget']:.2f}%, Test: {unlearned_accs['test']:.2f}%")
                print(f"    - Scores -> ToW: {tow_score:.3f}, Pred Diff: {pred_diff:.3f}%")

    # ===================================================================
    # 7. 결과 정리 및 출력
    # ===================================================================
    print(f"\n{'='*20} Final Results {'='*20}")
    
    def format_results(data):
        mean = np.mean(data)
        if len(data) < 2: return f"{mean:.3f}"
        sem = stats.sem(data)
        ci = sem * stats.t.ppf((1 + 0.95) / 2., len(data)-1)
        return f"{mean:.3f} ± {ci:.3f}"

    output_data = []
    es_levels = ["Low ES", "Medium ES", "High ES"]
    for method in all_methods:
        row = {'Method': method}
        for es in es_levels:
            row[f'ToW_{es}'] = format_results(results["ToW"][method][es])
        for es in es_levels:
            row[f'Pred_Diff_{es}'] = format_results(results["Pred_Diff"][method][es])
        output_data.append(row)

    df = pd.DataFrame(output_data)
    df.columns = pd.MultiIndex.from_tuples([
        ("", "Method"), ("ToW", "Low ES"), ("ToW", "Medium ES"), ("ToW", "High ES"),
        ("Percentage of different predictions (%)", "Low ES"),
        ("Percentage of different predictions (%)", "Medium ES"),
        ("Percentage of different predictions (%)", "High ES")
    ])

    print(df.to_string(index=False))

if __name__ == '__main__':
    main()

Using device: cuda
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified

==================== Starting Run 1/3 ====================

[TRAINING MODE] Step 1/5: Training original model...
    Epoch 1/30 completed in 31.16s
    Epoch 2/30 completed in 29.40s
    Epoch 3/30 completed in 28.80s
    Epoch 4/30 completed in 30.71s
    Epoch 5/30 completed in 29.65s
    Epoch 6/30 completed in 29.20s
    Epoch 7/30 completed in 32.04s
    Epoch 8/30 completed in 31.04s
    Epoch 9/30 completed in 29.13s
    Epoch 10/30 completed in 28.91s
    Epoch 11/30 completed in 28.80s
    Epoch 12/30 completed in 28.95s
    Epoch 13/30 completed in 28.69s
    Epoch 14/30 completed in 28.41s
    Epoch 15/30 completed in 28.46s
    Epoch 16/30 completed in 28.63s
    Epoch 17/30 completed in 29.01s
    Epoch 18/30 completed in 29.20s
    Epoch 19/30 completed in 29.31s
    Epoch 20/30 completed in 29.30s
    Epoch 21/30 completed in 29.14s
    Ep